In [ ]:
#upload the data into colab
import opendatasets as od
import pandas
  
od.download("https://www.kaggle.com/datasets/msambare/fer2013")

Skipping, found downloaded files in "./fer2013" (use force=True to force download)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Flatten, BatchNormalization, Activation, Dropout
import os
import tensorflow as tf

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
device = tf.config.list_physical_devices('GPU')
if len(device) > 0:
        tf.config.experimental.set_memory_growth(device[0], True)

In [ ]:
main_dir = 'fer2013'
train_dir = os.path.join(main_dir, 'train')
test_dir = os.path.join(main_dir, 'test')

class_names = ['angry', 'disgust', 'fear', 'happy', 'sad','surprise','neutral']

train_set = tf.keras.utils.image_dataset_from_directory(
    directory=train_dir,
    labels='inferred',
    label_mode='int',
    class_names=class_names,
    color_mode='rgb',
    batch_size=32,
    image_size=(48, 48),
    shuffle=True,
    seed=1024,
    validation_split=0.2,
    subset='training',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)
validation_set = tf.keras.utils.image_dataset_from_directory(
    directory=train_dir,
    labels='inferred',
    label_mode='int',
    class_names=class_names,
    color_mode='rgb',
    batch_size=32,
    image_size=(48, 48),
    shuffle=True,
    seed=1024,
    validation_split=0.2,
    subset='validation',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)
test_set = tf.keras.utils.image_dataset_from_directory(
    directory=test_dir,
    labels='inferred',
    label_mode='int',
    class_names=class_names,
    color_mode='rgb',
    batch_size=32,
    image_size=(48, 48),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 28709 files belonging to 7 classes.
Using 22968 files for training.
Found 28709 files belonging to 7 classes.
Using 5741 files for validation.
Found 7178 files belonging to 7 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Flatten, BatchNormalization, Activation, Dropout

with tf.device('/device:GPU:0'):
  NN_model = Sequential([
    InputLayer(input_shape=(48, 48, 3)),
    Flatten(),
    Dense(units = 256),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.25),
    Dense(512),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.25),
    Dense(units=7, activation = 'softmax')
])

In [ ]:
NN_model.build((48, 48, 3))
NN_model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_9 (Flatten)         (None, 6912)              0         
                                                                 
 dense_27 (Dense)            (None, 256)               1769728   
                                                                 
 batch_normalization_18 (Bat  (None, 256)              1024      
 chNormalization)                                                
                                                                 
 activation_18 (Activation)  (None, 256)               0         
                                                                 
 dropout_18 (Dropout)        (None, 256)               0         
                                                                 
 dense_28 (Dense)            (None, 512)               131584    
                                                      

In [ ]:
def input_image_normalization(image, label):
  image = tf.cast(image/255. , tf.float32)
  return image, label
norm_train_set = train_set.map(input_image_normalization)
norm_vald_set = validation_set.map(input_image_normalization)
norm_test_set = test_set.map(input_image_normalization)

In [ ]:
optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)
NN_model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

In [ ]:
history = NN_model.fit(norm_train_set, validation_data=norm_vald_set,
                        epochs=30, batch_size=32, 
                        shuffle=False)

Epoch 1/30
718/718 [==============================] - 11s 14ms/step - loss: 1.8888 - accuracy: 0.2830 - val_loss: 1.8361 - val_accuracy: 0.3310
Epoch 2/30
718/718 [==============================] - 10s 14ms/step - loss: 1.7541 - accuracy: 0.3230 - val_loss: 1.6905 - val_accuracy: 0.3424
Epoch 3/30
718/718 [==============================] - 10s 13ms/step - loss: 1.7012 - accuracy: 0.3419 - val_loss: 1.6755 - val_accuracy: 0.3364
Epoch 4/30
718/718 [==============================] - 10s 13ms/step - loss: 1.6570 - accuracy: 0.3619 - val_loss: 1.6972 - val_accuracy: 0.3478
Epoch 5/30
718/718 [==============================] - 9s 13ms/step - loss: 1.6218 - accuracy: 0.3724 - val_loss: 1.6718 - val_accuracy: 0.3593
Epoch 6/30
718/718 [==============================] - 10s 13ms/step - loss: 1.5863 - accuracy: 0.3853 - val_loss: 1.7082 - val_accuracy: 0.3473
Epoch 7/30
718/718 [==============================] - 10s 15ms/step - loss: 1.5589 - accuracy: 0.3950 - val_loss: 1.6354 - val_accuracy: 

In [ ]:
NN_model.evaluate(norm_test_set)

225/225 [==============================] - 3s 12ms/step - loss: 1.8071 - accuracy: 0.3644


[1.8070987462997437, 0.3644469082355499]